##**Import libraries**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from time import time

import pycuda.driver as drv
import pycuda.gpuarray as gpuarray

from pycuda.compiler import SourceModule

ModuleNotFoundError: No module named 'pycuda'

In [3]:
pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 11.9 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.1-cp311-cp311-linux_x86_64.whl size=660712 sha256=68e4b0ea9ddda38a929b573ab93ab1592af3a067d22c96e0ce5beb9995eeebff
  Stored in directory: /root/.cache/pip/wheels/49/0a/64/6530a5fde64f984ebb4992e38744fdfd2a61f510377b3a24d9
Successfully built pycuda


## **Get information of GPU connected**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jul  8 17:47:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P0             28W /   70W |     126MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

##**Set parameters**

In [ ]:
# --- Spatial & temporal discretization ---
dx = dy = 0.025
nx = ny = 500

dt      = 5e-8      # from FiPy example
stepmax = 10000     # match FiPy’s 10 000 steps

# --- Physical coefficients ---
tau    = 3e-4       # relaxation time
kappa  = 2.25       # thermal diffusivity

# --- Anisotropy (6‐fold) ---
zeta   = 0.02       # anisotropy strength
aniso  = 6.0        # symmetry order
angle0 = np.pi/8    # orientation

# --- Temperature‐coupling ---
kappa1 = 0.9
kappa2 = 20.0

# --- Seed & interface ---
r0    = 5 * dx      # radius = 5 cells
width = dx          # diffuse‐interface ≈ 1 cell


##**Define arrays in Host (CPU)**

In [ ]:
phi_host = np.zeros((nx,ny))
phi_new_host = np.zeros((nx,ny))
temp_host = np.zeros((nx,ny))
temp_new_host = np.zeros((nx,ny))
grad_phix_host = np.zeros((nx,ny))
grad_phiy_host = np.zeros((nx,ny))
a2_host = np.zeros((nx,ny))
lap_temp_host = np.zeros((nx,ny))
lap_phi_host = np.zeros((nx,ny))
ax_host = np.zeros((nx,ny))
ay_host = np.zeros((nx,ny))

## **Set initial distributions**

In [ ]:
# — Initialization in dimensionless units —

# Allocate & zero
phi_host  = np.zeros((nx,ny), dtype=np.float64)
temp_host = np.zeros((nx,ny), dtype=np.float64)

# Initialize a smooth circle of φ and matching temperature θ = –φ
for j in range(ny):
    y = (j/ny) - 0.5
    for i in range(nx):
        x = (i/nx) - 0.5
        r = np.hypot(x, y)
        phi_val = 0.5*(1.0 - np.tanh((r-r0)/width))
        phi_host[i,j]  = phi_val
        temp_host[i,j] = -0.5


## **Define "Device code" to calculate gradient of phi and temp and interfacial anisotropy**

In [ ]:
def get_kernel_string1_nd(nx, ny, dx, dy, pi, zeta, aniso, angle0):
    kernel = """
    #include <math.h>
    #define nx {nx}
    #define ny {ny}
    #define dx {dx:.6e}
    #define dy {dy:.6e}
    #define pi  {pi}
    #define zeta {zeta}
    #define aniso {aniso}
    #define angle0 {angle0}

    __global__ void calcgrad(
        const double *phi,
        const double *temp,
        double *grad_phix,
        double *grad_phiy,
        double *lap_phi,
        double *lap_temp,
        double *ax,
        double *ay,
        double *a2
    ){{
        int i = blockIdx.x * blockDim.x + threadIdx.x;
        int j = blockIdx.y * blockDim.y + threadIdx.y;
        if(i >= nx || j >= ny) return;
        int ip = (i + 1) % nx, im = (i - 1 + nx) % nx;
        int jp = (j + 1) % ny, jm = (j - 1 + ny) % ny;

        int idx    = j*nx + i;
        int idx_ip = j*nx + ip, idx_im = j*nx + im;
        int idx_jp = jp*nx + i, idx_jm = jm*nx + i;

        // gradients
        grad_phix[idx] = (phi[idx_ip] - phi[idx_im]) / (2.0 * dx);
        grad_phiy[idx] = (phi[idx_jp] - phi[idx_jm]) / (2.0 * dy);

        // 9-point laplacian (dimensionless)
        lap_phi[idx]  = (
            2.0*(phi[idx_ip] + phi[idx_im] + phi[idx_jp] + phi[idx_jm]) +
            phi[jp*nx+ip] + phi[jm*nx+im] + phi[jp*nx+im] + phi[jm*nx+ip] -
            12.0*phi[idx]
        ) / (3.0 * dx * dx);

        lap_temp[idx] = (
            2.0*(temp[idx_ip] + temp[idx_im] + temp[idx_jp] + temp[idx_jm]) +
            temp[jp*nx+ip] + temp[jm*nx+im] + temp[jp*nx+im] + temp[jm*nx+ip] -
            12.0*temp[idx]
        ) / (3.0 * dx * dx);

        // anisotropy angle ψ
        double gx = grad_phix[idx], gy = grad_phiy[idx];
        double ang;
        if (gx == 0.0) {{
            ang = (gy > 0.0) ? 0.5*pi : -0.5*pi;
        }} else if (gx > 0.0) {{
            ang = (gy >= 0.0) ? atan(gy/gx) : 2.0*pi + atan(gy/gx);
        }} else {{
            ang = pi + atan(gy/gx);
        }}

        // anisotropy strength a(ψ) and its derivative
        double a = 1.0 + zeta * cos(aniso*(ang - angle0));
        double da = -aniso*zeta * sin(aniso*(ang - angle0));

        // store flux‐coefficients
        ax[idx] =  a * da * gx;
        ay[idx] = -a * da * gy;
        a2[idx] =  a * a;
    }}
    """
    return kernel.format(
        nx=nx, ny=ny,
        dx=dx, dy=dy,
        pi=pi, zeta=zeta,
        aniso=aniso, angle0=angle0
    )

import numpy as np
from pycuda.compiler import SourceModule

# ensure block_size_string is defined
block_size_string = "#define block_size_x 16\n#define block_size_y 16\n"

# define π if not already
pi = np.pi
import pycuda.autoinit
from pycuda import driver as drv
device = drv.Context.get_current().get_device()
major, minor = device.compute_capability()
cc = f"{major}{minor}"
# gcc out your kernel string
kernel_string1 = get_kernel_string1_nd(
    nx, ny,
    dx, dy,
    pi,
    zeta,
    aniso,
    angle0
)

# compile
mod = SourceModule(block_size_string + kernel_string1, arch="sm_" + cc)
calcgrad = mod.get_function("calcgrad")
print("Compiled calcgrad:", calcgrad)


Compiled calcgrad: <pycuda._driver.Function object at 0x7abcc8d33740>


## **Define "Device code" to solve time evolution equations**

In [ ]:
from string import Template

def get_kernel_string2_nd(nx, ny, dx, dy, dt, tau, kappa, kappa1, kappa2):
    tmpl = Template(r"""
    #include <math.h>
    #define nx     $nx
    #define ny     $ny
    #define dx     $dx
    #define dy     $dy
    #define dt     $dt
    #define tau    $tau
    #define kappa  $kappa
    #define kappa1 $kappa1
    #define kappa2 $kappa2
    #define pi     3.141592653589793

    __global__ void timeevol(
        const double *phi,
        const double *temp,
        double       *phi_new,
        double       *temp_new,
        const double *ax,
        const double *ay,
        const double *a2,
        const double *grad_phix,
        const double *grad_phiy,
        const double *lap_phi,
        const double *lap_temp
    ){
        int i = blockIdx.x * blockDim.x + threadIdx.x;
        int j = blockIdx.y * blockDim.y + threadIdx.y;
        if(i >= nx || j >= ny) return;

        int ip = (i + 1) % nx, im = (i - 1 + nx) % nx;
        int jp = (j + 1) % ny, jm = (j - 1 + ny) % ny;

        int idx    = j*nx + i;
        int idx_ip = j*nx + ip, idx_im = j*nx + im;
        int idx_jp = jp*nx + i, idx_jm = jm*nx + i;

        // divergence of anisotropic flux D∇φ
        double d_ay_dx = (ay[idx_ip] - ay[idx_im]) / (2.0 * dx);
        double d_ax_dy = (ax[idx_jp] - ax[idx_jm]) / (2.0 * dy);
        double d_a2_dx = (a2[idx_ip] - a2[idx_im]) / (2.0 * dx);
        double d_a2_dy = (a2[idx_jp] - a2[idx_jm]) / (2.0 * dy);
        double div_flux = d_ay_dx + d_ax_dy
                        + a2[idx] * lap_phi[idx]
                        + d_a2_dx * grad_phix[idx]
                        + d_a2_dy * grad_phiy[idx];

        // Allen–Cahn reaction term with proper undercooling
        double xi     = phi[idx];
        double deltaT = -temp[idx];               // ΔT = -θ′
        double m      = (xi - 0.5)
                      - (kappa1/pi) * atan(kappa2 * deltaT);
        double source = xi * (1.0 - xi) * m;
        double dxi_dt = (div_flux + source) / tau;
        double phi_tmp = xi + dt * dxi_dt;
        if(phi_tmp < 0.0)      phi_tmp = 0.0;
        else if(phi_tmp > 1.0) phi_tmp = 1.0;
        phi_new[idx] = phi_tmp;

        // Temperature evolution: ∂θ′/∂t = κ ∇²θ′ + ∂φ/∂t
        double dT_dt = kappa * lap_temp[idx] + dxi_dt;
        double temp_tmp = temp[idx] + dt * dT_dt;
        // clamp θ′ into [−1,0]
        if(temp_tmp < -1.0)      temp_tmp = -1.0;
        else if(temp_tmp >  0.0) temp_tmp =  0.0;
        temp_new[idx] = temp_tmp;
    }
    """)
    return tmpl.substitute(
        nx=nx, ny=ny,
        dx=f"{dx:.6e}", dy=f"{dy:.6e}",
        dt=f"{dt:.6e}", tau=f"{tau:.6e}",
        kappa=f"{kappa:.6e}",
        kappa1=f"{kappa1:.6e}",
        kappa2=f"{kappa2:.6e}"
    )

# Recompile your kernel:
kernel_string2 = get_kernel_string2_nd(nx, ny, dx, dy, dt, tau, kappa, kappa1, kappa2)
timeevol_mod   = SourceModule(block_size_string + kernel_string2, arch="sm_" + cc)
timeevol       = timeevol_mod.get_function("timeevol")


/usr/local/lib/python3.11/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


## **Allocate device memory, data transfer, and execute device codes**

In [ ]:
# ─── Cell 1: PyCUDA + context setup ───
import numpy as np
import matplotlib.pyplot as plt

# Use autoinit so you never lose your context
import pycuda.autoinit
from pycuda import driver as drv
from pycuda.compiler import SourceModule
from time import time

# Query device once
device = drv.Context.get_current().get_device()
major, minor = device.compute_capability()
cc = f"{major}{minor}"           # e.g. "75" → arch="sm_75"

# Shared CUDA launch configuration
threads = (16, 16, 1)
grid    = (nx // threads[0], ny // threads[1], 1)
block_size_string = (
    "#define block_size_x 16\n"
    "#define block_size_y 16\n"
)

# Alias for explicit context sync if ever needed
context = drv.Context.get_current()


In [ ]:
# ─── Cell 2: Compile kernels ───
import numpy as np   # for pi, etc.

pi = np.pi

# Generate and compile calcgrad
kernel_string1 = get_kernel_string1_nd(nx, ny, dx, dy, pi, zeta, aniso, angle0)
calcgrad_mod   = SourceModule(block_size_string + kernel_string1, arch="sm_" + cc)
calcgrad       = calcgrad_mod.get_function("calcgrad")

# Generate and compile timeevol
kernel_string2 = get_kernel_string2_nd(nx, ny, dx, dy, dt, tau, kappa, kappa1, kappa2)
timeevol_mod   = SourceModule(block_size_string + kernel_string2, arch="sm_" + cc)
timeevol       = timeevol_mod.get_function("timeevol")

print("Kernels loaded:", calcgrad, timeevol)


Kernels loaded: <pycuda._driver.Function object at 0x7abcc8e524c0> <pycuda._driver.Function object at 0x7abcc98d83c0>


In [ ]:
# ─── Cell 3: Allocate + launch ───

# 1) Allocate GPU buffers
phi      = drv.mem_alloc(phi_host.nbytes)
…
a2       = drv.mem_alloc(a2_host.nbytes)

# 2) Upload data
drv.memcpy_htod(phi, phi_host)
drv.memcpy_htod(temp, temp_host)

# 3) Warm up & timing
context.synchronize()
start_evt.record()
t0 = time()

# … then your calcgrad/timeevol loop …

# 11) End timing
end_evt.record()
context.synchronize()
t1 = time()
print("Elapsed (ms):", (t1 - t0)*1000)


SyntaxError: invalid character '…' (U+2026) (ipython-input-45-361722956.py, line 5)

In [ ]:
# --- Debug: check final field values ---
print("φ′   min/max:", phi_result.min(), phi_result.max())
print("θ′   min/max:", temp_result.min(), temp_result.max())
